In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import random
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve 
import json

In [2]:
data = np.load('./data/data_scaled.npz')
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data["y_val"]
X_test = data['X_test']
y_test = data['y_test']

In [3]:

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rates=[0.5,0.25]):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)

        self.relu = nn.ReLU()
        self.input_dropout = nn.Dropout(dropout_rates[0])
        self.hidden_dropout = nn.Dropout(dropout_rates[1])

    def forward(self, x):
        x = self.input_dropout(x)
        x = self.relu(self.fc1(x))
        x = self.hidden_dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [4]:
def getPrAucIndividualClass(train_or_val, all_labels, all_preds, writer, epoch):
    # stack label arrays vertically
    all_labels = np.vstack(all_labels)
    all_preds = np.vstack(all_preds)
    # convert them to torch, for gpu compatibility
    all_labels = torch.tensor(all_labels)
    all_preds = torch.tensor(all_preds)
    all_pr_auc = []
    all_roc_auc = []
    # loop over 9 tasks
    for class_idx in range(all_labels.shape[1]):
        mask = ~torch.isnan(all_labels[:, class_idx])   # mask out NaN values
        class_labels = all_labels[:,class_idx][mask]    # get labels for current class
        class_preds = all_preds[:,class_idx][mask]      # get model preds for current class
        roc_auc = roc_auc_score(class_labels, class_preds)
        baseline = class_labels.mean()   # baseline is portion of pos labels  
        precision, recall, _ = precision_recall_curve(class_labels, class_preds)
        pr_auc = auc(recall, precision) - baseline # computes delta auc-pr
        all_pr_auc.append(pr_auc)
        all_roc_auc.append(roc_auc)

        # used for logging to tensorboard
        # writer.add_pr_curve(f"{train_or_val} Precision-Recall Class {class_idx}", class_labels, class_preds, epoch)
        # writer.add_scalar(f"{train_or_val} PR-AUC Delta Class {class_idx}", pr_auc, epoch)
        # writer.add_scalar(f"{train_or_val} ROC-AUC Class {class_idx}", roc_auc, epoch)
        
    # return metrics averaged over tasks and not averaged over tasks
    return np.mean(np.array(all_pr_auc)), np.mean(np.array(all_roc_auc)), np.array(all_pr_auc), np.array(all_roc_auc)

In [5]:
def compute_weighted_loss(y_pred, y_true):
    """
    Compute batch-wise weighted loss, giving equal influence to all targets.
    """
    # Create a mask for valid labels (non-NaN)
    mask = ~torch.isnan(y_true)  # True where valid

    # Count valid labels per target in the batch
    target_counts = mask.sum(dim=0)  # Number of valid samples per target in this batch
    # Avoid division by zero (replace 0 counts with 1)
    target_counts = target_counts.float().clamp(min=1)

    # Compute target-wise weights (Inverse of label frequency)
    target_weights = target_counts.max() / target_counts  # Higher weight for rare labels
    # Compute loss (Binary Cross-Entropy)
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
    loss = criterion(y_pred, torch.nan_to_num(y_true, nan=0.0))  # Replace NaNs with 0 for loss computation

    # Apply target-wise weights (broadcasting)
    weighted_loss = loss * target_weights

    # Compute final loss (only averaging over valid elements)
    final_loss = torch.nanmean(weighted_loss)  # Ignore NaNs in loss calculation
    return final_loss

# not used
def masked_bce_loss(outputs, targets):
    mask = ~torch.isnan(targets)  # Create a mask where values are NOT NaN
    loss = nn.functional.binary_cross_entropy_with_logits(outputs[mask], targets[mask])  # Compute loss only on valid labels
    return loss

In [6]:
def save_best_model(model, avg_pr_auc, best_score, file_path):
    # Check if the current average PR AUC is better than the best recorded score
    if avg_pr_auc > best_score:
        torch.save(model.state_dict(), file_path)
        print(f"Saved new best model with avg PR AUC: {avg_pr_auc:.4f}")
        return True
    else:
        print(f"Current avg PR AUC: {avg_pr_auc:.4f} did not improve over best score: {best_score:.4f}")
        return False

In [7]:
# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, device, writer, num_epochs=10):
    model.to(device)
    best_score = 0
    file_path = "best_model.pth"
    for epoch in range(num_epochs):
        # ------------- TRAINING ------------ #
        model.train()
        running_loss = 0.0
        all_labels_train = []
        all_preds_train = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.to(device)  # load inputs & labels onto gpu
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # criterion is loss fct passed to function, NaN values are masked there
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Store predictions & labels for AUC-PR computation
            all_labels_train.append(labels.detach().cpu().numpy())
            all_preds_train.append(torch.sigmoid(outputs).detach().cpu().numpy())  # Convert logits to probabilities

        # Compute training AUC-PR
        train_pr_auc, train_roc_auc, _, _ = getPrAucIndividualClass("Train", all_labels_train, all_preds_train, writer, epoch)
        epoch_loss_train = running_loss / len(train_loader)

        # log loss and auc-pr to tensorboard
        # writer.add_scalar("Loss/Training", epoch_loss_train, epoch)

        # --------- EVALUATION AFTER EPOCH --------------- #
        model.eval()
        val_loss = 0.0
        all_labels_val = []
        all_preds_val = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device) # load to gpu
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Store predictions & labels for AUC-PR computation
                all_labels_val.append(labels.cpu().numpy())
                all_preds_val.append(torch.sigmoid(outputs).cpu().numpy())  # Convert logits to probabilities

        epoch_loss_val = val_loss / len(val_loader)

        # log to tensorboard
        # writer.add_scalar("Loss/Validation", epoch_loss_val, epoch)

        # Compute Validation metrics
        val_pr_auc, val_roc_auc, _, _ = getPrAucIndividualClass("Val", all_labels_val, all_preds_val, writer, epoch)
        # if better, store model
        saved = save_best_model(model, avg_pr_auc=val_pr_auc, best_score=best_score, file_path=file_path)
        if saved:
            best_score = val_pr_auc

        # ------------------- END of EPOCH STUFF ------------- #
        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_loss_train:.4f}, Avg Train AUC-PR: {train_pr_auc:.4f}, Avg Train AUC-ROC: {train_roc_auc:.4f} | "
              f"Val Loss: {epoch_loss_val:.4f}, Avg Val AUC-PR: {val_pr_auc:.4f}, Avg Val AUC-ROC: {val_roc_auc:.4f}"
        )
        
        # Log model parameters and gradients to tensorboard
        # for name, param in model.named_parameters():
        #     writer.add_histogram(name, param, epoch)
        #     writer.add_histogram(f"{name}.grad", param.grad, epoch)
        # log model architecture to tensorboard
        # writer.add_graph(model, torch.randn(1, 2248).to(device))
    
    # ----------- EVALUATE BEST MODEL ------------- #
    model.load_state_dict(torch.load(file_path, map_location=device)) # load best model
    model.eval()
    val_loss = 0.0
    all_labels_val = []
    all_preds_val = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device) # load input & labels onto gpu
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Store predictions & labels for AUC-PR computation
            all_labels_val.append(labels.cpu().numpy())
            all_preds_val.append(torch.sigmoid(outputs).cpu().numpy())  # Convert logits to probabilities

    epoch_loss_val = val_loss / len(val_loader)
    #writer.add_scalar("Loss/Validation", epoch_loss_val, epoch) # tensorboard

    # Compute metrics for best model and return them
    val_pr_auc, val_roc_auc, val_pr_auc_per_task, val_roc_auc_per_task = getPrAucIndividualClass("Best Val", all_labels_val, all_preds_val, writer, epoch)
    print(f"Best Model Results | "
              f"Avg Val AUC-PR: {val_pr_auc:.4f}, Avg Val AUC-ROC: {val_roc_auc:.4f}"
        )
    return epoch_loss_val, val_pr_auc, val_roc_auc, val_pr_auc_per_task, val_roc_auc_per_task


In [8]:
# creae dataloaders
def get_datasets(X_train, y_train, X_val, y_val, X_test, y_test, seed):
    def worker_init_fn(worker_id):
        # Adjust the seed based on the worker id to ensure different seeds for each worker.
        np.random.seed(seed + worker_id)
        random.seed(seed + worker_id)

    batch_size = 120
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,worker_init_fn=worker_init_fn)

    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,worker_init_fn=worker_init_fn)

    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=worker_init_fn)

    return train_loader, val_loader, test_loader

In [10]:
# hyperparameters & global vars
input_size = 2248 
hidden_sizes = [120, 48, None]
output_size = 9
batch_size = 120
lr = 0.001
dropout_rates = [0.3, 0.5] # input dropout, hidden dropout
seeds = [8479, 227, 5413, 8179, 7528]

# list to collect results avg over task
avg_results = {
        "Delta-AUC-PR": [],
        "ROC-AUC": []
    }

# list to collect results per task
all_results = {
        "Delta-AUC-PR": [],
        "ROC-AUC": []
    }  

# train and evaluate model 5 times
for run in range(5):
    # set seed
    seed_value = seeds[run]
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    # initialize
    # writer = SummaryWriter(f"manual_runs/run_36_upsampling") # tensorboard
    writer = None
    model = NeuralNetwork(input_size, hidden_sizes, output_size, dropout_rates)
    criterion = compute_weighted_loss
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # train model
    train_loader, val_loader, test_loader = get_datasets(X_train, y_train, X_val, y_val, X_test, y_test, seed_value)
    val_loss, val_delta_auc_pr, val_roc_auc, val_pr_auc_per_task, val_roc_auc_per_task = train_model(model, train_loader, test_loader, criterion, optimizer, device, writer, num_epochs=20)
    
    # collect results
    avg_results["Delta-AUC-PR"].append(val_delta_auc_pr)
    avg_results["ROC-AUC"].append(val_roc_auc)

    all_results["Delta-AUC-PR"].append(np.array(val_pr_auc_per_task))
    all_results["ROC-AUC"].append(np.array(val_roc_auc_per_task))
    
    # ------- for hyperparameter tuning ------- #
    # val_loss, val_delta_auc_pr, val_roc_auc = train_model(model, train_loader, val_loader, criterion, optimizer, device, writer, num_epochs=20)

    # hparams = {
    #     "lr": lr,
    #     "batch_size": batch_size,
    #     "optimizer": "Adam",
    #     "hidden_sizes_1": hidden_sizes[0],
    #     "hidden_sizes_2": hidden_sizes[1]
    # }
    # metrics = {
    #     "auc-pr": val_delta_auc_pr,
    #     "loss": val_loss
    # }
    # writer.add_hparams(hparams, metrics)
    # writer.close()


cuda
Saved new best model with avg PR AUC: 0.0391
Epoch 1/20 | Train Loss: 0.0446, Avg Train AUC-PR: 0.0008, Avg Train AUC-ROC: 0.5493 | Val Loss: 0.0059, Avg Val AUC-PR: 0.0391, Avg Val AUC-ROC: 0.6500
Saved new best model with avg PR AUC: 0.0513
Epoch 2/20 | Train Loss: 0.0037, Avg Train AUC-PR: 0.0142, Avg Train AUC-ROC: 0.7350 | Val Loss: 0.0057, Avg Val AUC-PR: 0.0513, Avg Val AUC-ROC: 0.7398
Current avg PR AUC: 0.0294 did not improve over best score: 0.0513
Epoch 3/20 | Train Loss: 0.0031, Avg Train AUC-PR: 0.0205, Avg Train AUC-ROC: 0.8637 | Val Loss: 0.0053, Avg Val AUC-PR: 0.0294, Avg Val AUC-ROC: 0.7145
Current avg PR AUC: 0.0149 did not improve over best score: 0.0513
Epoch 4/20 | Train Loss: 0.0030, Avg Train AUC-PR: 0.0392, Avg Train AUC-ROC: 0.8886 | Val Loss: 0.0060, Avg Val AUC-PR: 0.0149, Avg Val AUC-ROC: 0.6683
Current avg PR AUC: 0.0284 did not improve over best score: 0.0513
Epoch 5/20 | Train Loss: 0.0030, Avg Train AUC-PR: 0.0681, Avg Train AUC-ROC: 0.9007 | Val L

In [11]:
# compute avg and sd over five runs
final_auc_pr = np.mean(np.array(avg_results["Delta-AUC-PR"]))
final_roc_auc = np.mean(np.array(avg_results["ROC-AUC"]))
final_sd_auc_pr = np.std(np.array(avg_results["Delta-AUC-PR"]))
final_sd_roc_auc = np.std(np.array(avg_results["ROC-AUC"]))

# save in json convertible format
final_results = [{
    "Delta-AUC-PR": float(final_auc_pr),
    "ROC-AUC": float(final_roc_auc),
    "Sd-Delta-AUC-PR": float(final_sd_auc_pr),
    "Sd-ROC-AUC": float(final_sd_roc_auc)
}]
final_results

# save to json file
with open("./metrics/metrics_pretraining.json", "w") as f:
    json.dump(final_results, f, indent=4)

In [ ]:
# calculate mean and sd for five runs but don't average over tasks
avg_auc_pr_per_run = np.mean(np.array(all_results["Delta-AUC-PR"]), axis=0)
sd_auc_pr_per_run = np.std(np.array(all_results["Delta-AUC-PR"]), axis=0)

avg_roc_auc_per_run = np.mean(np.array(all_results["ROC-AUC"]), axis=0)
sd_roc_auc_per_run = np.std(np.array(all_results["ROC-AUC"]), axis=0)

# save in json convertible format
final_results_per_task = [{
    "Delta-AUC-PR per task": list(avg_auc_pr_per_run),
    "ROC-AUC per task": list(avg_roc_auc_per_run),
    "Sd-Delta-AUC-PR per task": list(sd_auc_pr_per_run),
    "Sd-ROC-AUC per task": list(sd_roc_auc_per_run)
}]
final_results_per_task

# write to json file
with open("./metrics/metrics_pretraining_per_task.json", "w") as f:
    json.dump(final_results_per_task, f, indent=4)

TypeError: Object of type float32 is not JSON serializable

In [19]:
# calculate mean and sd for five runs but do average over tasks
avg_auc_pr_per_task = np.mean(np.array(all_results["Delta-AUC-PR"]), axis=1)
sd_auc_pr_per_task = np.std(np.array(all_results["Delta-AUC-PR"]), axis=1)

avg_roc_auc_per_task = np.mean(np.array(all_results["ROC-AUC"]), axis=1)
sd_roc_auc_per_task = np.std(np.array(all_results["ROC-AUC"]), axis=1)

# save in json convertible format
final_results_per_task = [{
    "Delta-AUC-PR per run": list(avg_auc_pr_per_task),
    "ROC-AUC per run": list(avg_roc_auc_per_task),
    "Sd-Delta-AUC-PR per run": list(sd_auc_pr_per_task),
    "Sd-ROC-AUC per run": list(sd_roc_auc_per_task)
}]
final_results_per_task

# write to json file
with open("./metrics/metrics_pretraining_per_run.json", "w") as f:
    json.dump(final_results_per_task, f, indent=4)

TypeError: Object of type float32 is not JSON serializable